# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 2 2022 1:13PM,238669,20,ACI-7970839-FLS-BO-FLS,"ACI Healthcare USA, Inc.",Released
1,Jun 2 2022 1:13PM,238669,20,8061467-BO,"ACI Healthcare USA, Inc.",Released
2,Jun 2 2022 1:13PM,238669,20,ACI-8091205,"ACI Healthcare USA, Inc.",Released
3,Jun 2 2022 1:13PM,238669,20,ACI-8091208,"ACI Healthcare USA, Inc.",Released
4,Jun 2 2022 1:13PM,238669,20,ACI-8091831,"ACI Healthcare USA, Inc.",Released
5,Jun 2 2022 1:13PM,238669,20,8091863,"ACI Healthcare USA, Inc.",Released
6,Jun 2 2022 1:13PM,238669,20,8091866,"ACI Healthcare USA, Inc.",Released
7,Jun 2 2022 1:13PM,238669,20,8091869,"ACI Healthcare USA, Inc.",Released
8,Jun 2 2022 1:13PM,238669,20,8091873,"ACI Healthcare USA, Inc.",Released
9,Jun 2 2022 1:13PM,238669,20,8091874,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
45,238665,Released,5
46,238666,Released,8
47,238667,Released,9
48,238668,Released,14
49,238669,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238665,NaN,NaN,5.0
238666,NaN,NaN,8.0
238667,NaN,NaN,9.0
238668,NaN,NaN,14.0
238669,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238553,0.0,1.0,0.0
238556,0.0,0.0,2.0
238557,0.0,0.0,5.0
238564,0.0,7.0,0.0
238578,4.0,1.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238553,0,1,0
238556,0,0,2
238557,0,0,5
238564,0,7,0
238578,4,1,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238553,0,1,0
1,238556,0,0,2
2,238557,0,0,5
3,238564,0,7,0
4,238578,4,1,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238553,,1,
1,238556,,,2
2,238557,,,5
3,238564,,7,
4,238578,4,1,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 2 2022 1:13PM,238669,20,"ACI Healthcare USA, Inc."
11,Jun 2 2022 1:09PM,238668,10,Beach Products Inc
25,Jun 2 2022 1:09PM,238667,20,"ACI Healthcare USA, Inc."
34,Jun 2 2022 1:06PM,238666,10,"Methapharm, Inc."
42,Jun 2 2022 1:06PM,238665,10,"Methapharm, Inc."
47,Jun 2 2022 1:06PM,238664,10,"Methapharm, Inc."
72,Jun 2 2022 12:51PM,238663,20,"ACI Healthcare USA, Inc."
83,Jun 2 2022 12:49PM,238659,10,Bio-PRF
84,Jun 2 2022 12:48PM,238661,12,Hush Hush
92,Jun 2 2022 12:47PM,238662,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 2 2022 1:13PM,238669,20,"ACI Healthcare USA, Inc.",,,11
1,Jun 2 2022 1:09PM,238668,10,Beach Products Inc,,,14
2,Jun 2 2022 1:09PM,238667,20,"ACI Healthcare USA, Inc.",,,9
3,Jun 2 2022 1:06PM,238666,10,"Methapharm, Inc.",,,8
4,Jun 2 2022 1:06PM,238665,10,"Methapharm, Inc.",,,5
5,Jun 2 2022 1:06PM,238664,10,"Methapharm, Inc.",,,25
6,Jun 2 2022 12:51PM,238663,20,"ACI Healthcare USA, Inc.",,,11
7,Jun 2 2022 12:49PM,238659,10,Bio-PRF,,,1
8,Jun 2 2022 12:48PM,238661,12,Hush Hush,,,8
9,Jun 2 2022 12:47PM,238662,10,"Nextsource Biotechnology, LLC",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 1:13PM,238669,20,"ACI Healthcare USA, Inc.",11,,
1,Jun 2 2022 1:09PM,238668,10,Beach Products Inc,14,,
2,Jun 2 2022 1:09PM,238667,20,"ACI Healthcare USA, Inc.",9,,
3,Jun 2 2022 1:06PM,238666,10,"Methapharm, Inc.",8,,
4,Jun 2 2022 1:06PM,238665,10,"Methapharm, Inc.",5,,
5,Jun 2 2022 1:06PM,238664,10,"Methapharm, Inc.",25,,
6,Jun 2 2022 12:51PM,238663,20,"ACI Healthcare USA, Inc.",11,,
7,Jun 2 2022 12:49PM,238659,10,Bio-PRF,1,,
8,Jun 2 2022 12:48PM,238661,12,Hush Hush,8,,
9,Jun 2 2022 12:47PM,238662,10,"Nextsource Biotechnology, LLC",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 1:13PM,238669,20,"ACI Healthcare USA, Inc.",11,,
1,Jun 2 2022 1:09PM,238668,10,Beach Products Inc,14,,
2,Jun 2 2022 1:09PM,238667,20,"ACI Healthcare USA, Inc.",9,,
3,Jun 2 2022 1:06PM,238666,10,"Methapharm, Inc.",8,,
4,Jun 2 2022 1:06PM,238665,10,"Methapharm, Inc.",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 1:13PM,238669,20,"ACI Healthcare USA, Inc.",11.0,NaN,NaN
1,Jun 2 2022 1:09PM,238668,10,Beach Products Inc,14.0,NaN,NaN
2,Jun 2 2022 1:09PM,238667,20,"ACI Healthcare USA, Inc.",9.0,NaN,NaN
3,Jun 2 2022 1:06PM,238666,10,"Methapharm, Inc.",8.0,NaN,NaN
4,Jun 2 2022 1:06PM,238665,10,"Methapharm, Inc.",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 2 2022 1:13PM,238669,20,"ACI Healthcare USA, Inc.",11.0,0.0,0.0
1,Jun 2 2022 1:09PM,238668,10,Beach Products Inc,14.0,0.0,0.0
2,Jun 2 2022 1:09PM,238667,20,"ACI Healthcare USA, Inc.",9.0,0.0,0.0
3,Jun 2 2022 1:06PM,238666,10,"Methapharm, Inc.",8.0,0.0,0.0
4,Jun 2 2022 1:06PM,238665,10,"Methapharm, Inc.",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3341041,98.0,15.0,4.0
12,1431674,16.0,4.0,0.0
15,1431714,13.0,41.0,15.0
16,954490,2.0,2.0,0.0
19,715952,13.0,0.0,0.0
20,1670378,33.0,8.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3341041,98.0,15.0,4.0
1,12,1431674,16.0,4.0,0.0
2,15,1431714,13.0,41.0,15.0
3,16,954490,2.0,2.0,0.0
4,19,715952,13.0,0.0,0.0
5,20,1670378,33.0,8.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,98.0,15.0,4.0
1,12,16.0,4.0,0.0
2,15,13.0,41.0,15.0
3,16,2.0,2.0,0.0
4,19,13.0,0.0,0.0
5,20,33.0,8.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,98.0
1,12,Released,16.0
2,15,Released,13.0
3,16,Released,2.0
4,19,Released,13.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Completed,4.0,0.0,15.0,0.0,0.0,0.0
Executing,15.0,4.0,41.0,2.0,0.0,8.0
Released,98.0,16.0,13.0,2.0,13.0,33.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Completed,4.0,0.0,15.0,0.0,0.0,0.0
1,Executing,15.0,4.0,41.0,2.0,0.0,8.0
2,Released,98.0,16.0,13.0,2.0,13.0,33.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Completed,4.0,0.0,15.0,0.0,0.0,0.0
1,Executing,15.0,4.0,41.0,2.0,0.0,8.0
2,Released,98.0,16.0,13.0,2.0,13.0,33.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()